In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
def load_dataset():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32')
    X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32')
    X_train = X_train / 255.0
    X_test = X_test / 255.0
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    return X_train, X_test, y_train, y_test

In [3]:
model = load_model('digitRecognizer.h5')

In [4]:
X_train, X_test, y_train, y_test = load_dataset()
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.2%}")

Test Loss: 0.0234, Test Accuracy: 99.28%


In [5]:
def load_new_image(path):
    newImage = load_img(path, color_mode='grayscale', target_size=(28, 28))
    newImage = img_to_array(newImage)
    newImage = newImage.reshape(1, 28, 28, 1)  # Reshape for the model
    newImage = newImage / 255.0
    return newImage

In [6]:
def test_model_performance(path):
    img = load_new_image(path)
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions, axis=-1)
    print(f'Predicted class: {predicted_class[0]}')

In [9]:
test_model_performance('sample_images\Test_image.png')

Predicted class: 5
